In [1]:
import numpy as np 
import pandas as pd 
import math 
from sklearn import preprocessing
import gmplot
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Masking, Input, Dropout
from keras.layers.merge import concatenate
from keras.utils import plot_model

Using TensorFlow backend.


In [2]:
dfs={}
for i in range(19):
	year=str(2000+i)
	file='datos/DesmatamentoMunicipios' + year + '.csv'
	dfs[i] = pd.read_csv(file, encoding = "ISO-8859-1", index_col=0, sep=",")
	#eliminamos columnas irrelevantes
	dfs[i].drop(columns='Latgms', inplace=True)
	dfs[i].drop(columns='Longms', inplace=True)
	dfs[i].drop(columns='CodIbge', inplace=True)
	#df.drop(columns='Soma', inplace=True)
	#df.rename(columns={0:'Latitud', 1:'Longitud', 2:'Municipio', 3:'Estado', 4: 'AreaKm2', 5:'Deforestacion', 6:'Incremento deforestacion', 7:'Bosque', 8:'Nubes', 9:'No observado', 10:'No bosque', 11:'Hidrografia', 12:'Check'} ,inplace=True)
	dfs[i].columns=['Latitud', 'Longitud', 'Municipio', 'Estado', 'Area total', 'Deforestacion ' + year, 'Incremento deforestacion ' + year, 'Bosque ' + year, 'Nubes ' + year, 'No observado ' + year, 'No bosque', 'Hidrografia', 'Check ' + year]
	dfs[i]=dfs[i][['Latitud', 'Longitud', 'Municipio', 'Estado', 'Area total', 'No bosque', 'Hidrografia', 'Deforestacion ' + year, 'Incremento deforestacion ' + year, 'Bosque ' + year, 'Nubes ' + year, 'No observado ' + year, 'Check ' + year]]
    


In [3]:
df=dfs[0]
dic={}
for idx, row in df.iterrows():
		if row['Municipio'] in dic:
			dic[row['Municipio']]+=1
		else:
			dic[row['Municipio']]=1

repetidos=[]
for key, value in dic.items():
	if value==2:
		repetidos.append(key)

rep= df.Municipio.isin(repetidos)
mismo_municipio =  df[rep]
print("Hay algunos municipios con el mismo nombre en 2 estados distintos:")
print(mismo_municipio.loc[:,'Latitud':'Area total'])

df=dfs[0]
for idx, value in dfs.items():
	if idx > 0:
		value.drop(columns='Latitud', inplace=True)
		value.drop(columns='Longitud', inplace=True)
		#value.drop(columns='Estado', inplace=True)
		value.drop(columns='Area total', inplace=True)
		value.drop(columns='Hidrografia', inplace=True)
		value.drop(columns='No bosque', inplace=True)
		df=pd.merge(df, value, on=['Municipio', 'Estado'])
		pd.set_option('display.max_columns', None)

print("Por eso, hacemos merge por la dupla municipio-estado:")
print(df.head())

Hay algunos municipios con el mismo nombre en 2 estados distintos:
      Latitud  Longitud               Municipio Estado  Area total
Nr                                                                
69   -9.99527 -68.15925              Rio Branco     AC        9113
104 -11.19374 -61.84743       Presidente Médici     RO        1773
105  -5.02798 -48.77112  Bom Jesus do Tocantins     PA        2827
170  -7.55219 -48.99658              Pau D'Arco     TO        1381
243  -7.72486 -50.13413              Pau D'Arco     PA        1674
264  -6.73568 -48.55630                Araguanã     TO         838
345 -15.28141 -58.15176              Rio Branco     MT         529
436  -2.30799 -45.76410       Presidente Médici     MA         441
626  -9.00182 -47.86311  Bom Jesus do Tocantins     TO        1338
636  -3.06101 -45.77225                Araguanã     MA         811
Por eso, hacemos merge por la dupla municipio-estado:
    Latitud  Longitud           Municipio Estado  Area total  No bosque  \


In [4]:
print("El dataset viene ordenado por deforestacion, de mayor a menor. Para entrenar el modelo alteramos el orden de forma aleatoria.")
df = df.sample(frac=1)#.reset_index(drop=True)
print(df.head())

El dataset viene ordenado por deforestacion, de mayor a menor. Para entrenar el modelo alteramos el orden de forma aleatoria.
      Latitud  Longitud            Municipio Estado  Area total  No bosque  \
481 -12.11997 -50.36955  Formoso do Araguaia     TO       13453    12257.6   
449  -5.31119 -48.49571          Esperantina     TO         505      227.5   
607  -2.98836 -45.42901     Pedro do Rosário     MA        1770       69.4   
650  -1.94722 -44.57133               Cedral     MA         291        0.2   
267  -4.72821 -45.55246       Marajá do Sena     MA         881        0.0   

     Hidrografia  Deforestacion 2000  Incremento deforestacion 2000  \
481        167.8               158.4                            NaN   
449         49.4               196.9                            NaN   
607          0.0                14.1                            NaN   
650          6.8                 0.1                            NaN   
267          0.0               701.3              

In [5]:
print("El area total cubierta por los municipios es: " + str(df.sum(axis = 0, skipna = True)[4]) + " km^2, mientras el area total de la Amazonia es de 5,5M km^2.")
print("Por ello, podemos afirmar que estamos cubriendo una superficie que supone el 92% del total.")

print("Dibujamos el mapa de " + str(len(df['Latitud'])) + " municipios")
gmap = gmplot.GoogleMapPlotter(df['Latitud'][0], df['Longitud'][0], 100) # coordenadas del primer municipio del dataframe
gmap.heatmap(df['Latitud'], df['Longitud']) 
gmap.draw("mapa.html")
print("Mapa de calor generado")

El area total cubierta por los municipios es: 5068048 km^2, mientras el area total de la Amazonia es de 5,5M km^2.
Por ello, podemos afirmar que estamos cubriendo una superficie que supone el 92% del total.
Dibujamos el mapa de 760 municipios
Mapa de calor generado


In [6]:
fixed_vars = df.loc[:,['Latitud', 'Longitud', 'Area total', 'Hidrografia', 'No bosque', 'Municipio', 'Estado']]
temporal_vars = df.loc[:,'Deforestacion 2000':]

# ensure all data is float
temporal_vars = temporal_vars.astype('float32')

# data structure for LSTM
# normalize features
print(temporal_vars.head())
x = temporal_vars.values[:,0:-6]
y = temporal_vars.values[:,-6].reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
x = scaler.fit_transform(x)
scaler2 = MinMaxScaler(feature_range=(0, 1))
y = scaler2.fit_transform(y)
# structure in arrays to be the input of the LSTM
vars_lstm=[]
municipio_len=len(x[0])
for mun in x:
	municipio=[]
	j=0
	while j < municipio_len:
		m=mun[j:j+6]
		if j==0:
			m[1]=-1		# marking missing values: Incremento deforestacion 1999/2000
		#elif m[5]!=municipio[0][5] or m[6]!=municipio[0][6]: # Hidrografia y No bosque son variables fijas!!!
		#	print("Hidrografia y No bosque varian de año en año")
		municipio.append(mun[j:j+6])
		j+=6
	vars_lstm.append(municipio)

vars_lstm=np.array(vars_lstm)
X1 = vars_lstm

# data structure for Dense
# One hot encoding 
fixed_vars = pd.concat([fixed_vars,pd.get_dummies(fixed_vars['Estado'])],axis=1)
# Drop column as it is now encoded
fixed_vars = fixed_vars.drop('Estado',axis = 1)
fixed_vars = fixed_vars.drop('Municipio',axis = 1)
print(fixed_vars.head())
scaler3 = MinMaxScaler(feature_range=(0, 1))
x = scaler3.fit_transform(fixed_vars)
X2 = np.array(x)

     Deforestacion 2000  Incremento deforestacion 2000  Bosque 2000  \
481          158.399994                            NaN   497.500000   
449          196.899994                            NaN    31.200001   
607           14.100000                            NaN  1686.500000   
650            0.100000                            NaN   283.799988   
267          701.299988                            NaN   179.699997   

     Nubes 2000  No observado 2000  Check 2000  Deforestacion 2001  \
481         0.0         371.700012       100.0          166.399994   
449         0.0           0.000000       100.0          209.100006   
607         0.0           0.000000       100.0         1647.400024   
650         0.0           0.100000       100.0          174.500000   
267         0.0           0.000000       100.0          708.400024   

     Incremento deforestacion 2001  Bosque 2001  Nubes 2001  \
481                       8.100000   489.500000         0.0   
449                      1

C:\Users\Luis\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:364: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\Luis\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:365: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
C:\Users\Luis\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [7]:
print("Ejemplo de input del modelo:")
print(X1[0], X2[0], y[0])

Ejemplo de input del modelo:
[[ 2.1962481e-02 -1.0000000e+00  3.2727337e-03  0.0000000e+00
   3.2832500e-02  9.0090036e-03]
 [ 1.9065732e-02  4.4786027e-03  3.2320064e-03  0.0000000e+00
   3.2832500e-02  9.0090036e-03]
 [ 1.6781410e-02  1.0272618e-03  3.2379115e-03  0.0000000e+00
   3.2832500e-02  9.0090036e-03]
 [ 1.5135307e-02  2.6555387e-03  3.2278064e-03  0.0000000e+00
   3.2832500e-02  0.0000000e+00]
 [ 1.3958592e-02  1.6628176e-03  3.2317520e-03  0.0000000e+00
   3.2832500e-02  9.0090036e-03]
 [ 1.2962171e-02  4.2619691e-03  3.2081117e-03  0.0000000e+00
   3.2832500e-02  9.0090036e-03]
 [ 1.2407646e-02  2.3625146e-03  3.2978770e-03  0.0000000e+00
   3.2832500e-02  0.0000000e+00]
 [ 1.1760822e-02  1.0256410e-03  3.2427586e-03  0.0000000e+00
   3.2426752e-02  9.0090036e-03]
 [ 1.1341338e-02  2.8754412e-03  3.2972586e-03  0.0000000e+00
   6.3706681e-02  9.0090036e-03]
 [ 1.1140740e-02  3.8253828e-03  6.0442192e-03  0.0000000e+00
   5.9843170e-03  1.0000000e+00]
 [ 1.0985456e-02  3.6

In [ ]:
def create_LSTM(mask=1):
	model = Sequential()
	if mask==0:
		model.add(LSTM(5, input_shape=(18, 6), activation='relu'))
	else:
		model.add(Masking(mask_value=-1, input_shape=(18,6)))
		model.add(LSTM(5, activation='relu'))
	model.add(Dense(16, activation='relu'))
	model.add(Dense(1, activation='relu'))
	model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['mae'])
	# fit model
	model.fit(X1[0:600], y[0:600], epochs=10, batch_size=1, verbose=2, validation_data=(X1[600:700], y[600:700]))
	# evaluate model on new data
	y_pred = model.predict(X1[700:-1])
	return y_pred

y_pred=create_LSTM(1)

In [ ]:
def create_model(mask=1):
    # LSTM para variables temporales
    input_temporal = Input(shape=(18, 6))
    if mask==0:
        lstm = LSTM(5, activation='relu')(input_temporal)
    else:
        masking = Masking(mask_value=-1)(input_temporal)
        lstm = LSTM(5, activation='relu')(masking)
    drop = Dropout(0.2)(lstm)
    dense_1 = Dense(16, activation = 'relu')(drop)
    dense_2 = Dense(16, activation = 'relu')(dense_1)
    
    # Dense para variables fijas
    input_fijo = Input(shape=(14,))
    dense_3 = Dense(32, activation = 'relu')(input_fijo)
    dense_4 = Dense(16, activation = 'relu')(dense_3)
    
    # concateno las 2 redes
    merge = concatenate([dense_2, dense_4])
    dense_5 = Dense(32, activation = 'relu')(merge)
    dense_6 = Dense(16, activation = 'relu')(dense_5)
    output = Dense(1, activation = 'relu')(dense_6)
    model = Model(inputs=[input_temporal, input_fijo], outputs=output)
    
    # summarize layers
    print(model.summary())
    # plot graph
    #plot_model(model, to_file='multiple_inputs.png')
    return model

model = create_model()
    
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['mae'])
# fit model
model.fit([X1[0:600], X2[0:600]], y[0:600], epochs=10, batch_size=1, verbose=2, validation_data=([X1[600:700], X2[600:700]], y[600:700]))


In [ ]:
# evaluate model on new data
y_pred = model.predict([X1[700:-1], X2[700:-1]])

In [ ]:
for i in range(len(y_pred)):
	print('Expected', y[i], 'Predicted', y_pred[i])


In [ ]:
# rescaling
y=scaler2.inverse_transform(y)
y_pred=scaler2.inverse_transform(y_pred)

In [ ]:
errores = []
num_test = len(y[700:-1])
for i in range(num_test):
    error = abs(y[i+700] - y_pred[i])
    print('Expected', y[i+700], 'Predicted', y_pred[i], 'Error', str(error))
    errores.append(error)
    if error > 1000:
        print (error)
print('Error absoluto medio', np.array(errores).mean())